![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



# Collecting Data from Plants with Phidgets

This is a utility notebook to collect data from [Phidgets sensors](https://www.phidgets.com/education/learn/projects/plant-kit/) around a plant and store in an online spreadsheet (Google Sheets).

We have tried to make this notebook as friendly as possible for beginner programmers. However, it does require some attention to detail, so follow the instructions carefully. 

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

## Overview

The purpose of this notebook is to collect data from certain Phidgets hardware sensors called which are set up to monitor the environment of a plant. The data can be viewed directly and it is also saved in an online spreadsheet for later data analysis.

<div align="center">
<img src="images/plant2.jpg" alt="A plant with sensor" width="400"/><br>
A basement window with our plant.
</div>

There are important requirements you must meet in order to complete this setup:
    
- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- you will need a computer (Mac, Windows, Raspberry Pi) with a USB connection and internet access
- you will need the Chrome or Chromium web browser (Safari, Firefox, and Opera will not work)

For a step by step guide on how to assemble your Phidgets plant kit , see the following [link](https://www.phidgets.com/education/learn/projects/plant-kit/assemble/)

There are five main steps in this notebook. 

1. Setting up the gauges 
2. Setting up an online spreadsheet
3. Connect the plant sensors to your computer
4. Setup code to post and display data
5. Start the timer, to continuously post data

***
## Step 1. Setting up the gauges

Run the cells in this notebook, **one cell at a time**. This will give you the chance to respond to any errors, and to chose a unique name for your online storage. 

**Do not select "Run All."**

We first set up some gauges to display values for temperature, humidity, soil moisture and light levels. We also add a text box to display the data as it is read in, and another to show the results of posting to the spreadsheet.  

We first load in all the libraries we need. Select the following cell and click the `▶Run` button.

In [ ]:
import json, pytz, os.path, random, re, requests, threading
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from time import sleep
from datetime import datetime
from IPython.display import display, Javascript, Markdown 

Now we set up the gauges and text boxes. This is not live data quite yet but we will be soon.

In [ ]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

# a text widget to monitor the latest data
data_monitor = widgets.Text(
    description='Posted data:',
    value="No posted data yet",
    layout={'width':'500px'},
    disabled = False
)
# a text widget to monitor results of posting to the cloud
post_monitor = widgets.Text(
    description='Post result:',
    value="No post yet",
    layout={'width':'500px'},
    disabled = False
)

# the watering button
data_button = widgets.Button(
    description='Update gauges',
    tooltip='Update data to gauges',
    disabled=False,
    button_style='success',
)

# the auto-watering Start/Stop button
post_button = widgets.Button(
    description='Post to spreadsheet',
    tooltip='Post the latest data to the spreadsheet',
    disabled=False,
    button_style='success',
)
    
dashboard = widgets.VBox([gauges,widgets.HBox([data_button,data_monitor]),
                              widgets.HBox([post_button,post_monitor])])

display(dashboard)

***
## Step 2: Setting up the online spreadsheet.

The next step involves creating an online spreadsheet using a web resource called Google Sheets. It's important to note that Google Sheets is a publicly accessible online spreadsheet tool, so please **avoid storing any confidential or sensitive data in it**.

In the following code cell, you'll find a random spreadsheet number which you should use throughout the Plant notebooks. If you've used this notebook before, the code may suggest using the same name/number as your previous sheet, but you can always enter a new name/number in the text box provide but again remember that this is publicly accessible online.

In [ ]:
file_name = './ss_name.txt'  # the file where we store the spreadsheet name

def get_ss_name():
    if (os.path.isfile(file_name)):
        with open(file_name,'r') as file:
            result=json.load(file)
        if result['name'] != 'default':
            return result
    json_name = {"name" : "Plant_" + "".join(random.choices('0123456789', k=6))}
    with open(file_name,'w') as file:
        json.dump(json_name,file)
        return json_name
    
def set_ss_name(name):
    with open(file_name,'w') as file:
        json.dump({"name" : name},file)
    
def f(SS_Name):
    if len(SS_Name) == 0:
        return "Enter a name here for your spreadsheet."
    if re.search(r'[^A-Za-z0-9_\-]',SS_Name):
        return "ERROR: Use only letters and numbers, no spaces."
    else:
        set_ss_name(SS_Name)
        return "Spreadsheet name is " + SS_Name

print("Choose a name for your spreadsheet here:")
widgets.interact(f, SS_Name=get_ss_name()['name']);

### 2a. Initializing the spreadsheet

Once you have selected a name above, we store some test data to ensure that it works. 

The following code runs a google script that takes some data and posts it online. The http link that is printed out will take you to your spreadsheet that holds the data. We also save the link in a file for later use.

In [ ]:
sheet_url = 'https://script.google.com/macros/s/AKfycbyuMjh718bkYDYCetSGb0gxDqWxxoK_JiwEKcNkUAKdOOfLHj6uWTlsnbkM_01I-RPIlw/exec'
sheet_name = get_ss_name()['name']

data = {'Date':'2023-09-01', 
                'Time':'09:00:00', 
                'Temperature':21, 
                'Humidity':50, 
                'Moisture':.5, 
                'Luminance':100 }


r = requests.post(sheet_url, params={'sheet':sheet_name, 'data':json.dumps(data)})

viewing_url = r.text[13:]
file_link = './ss_link.txt'  # the file where we store the spreadsheet link
json_link = {"link" : viewing_url }
with open(file_link,'w') as file:
    json.dump(json_link,file)

print(r.text)

### 2b. Viewing the spreadsheet

Your spreadsheet is saved online as a Google spreadsheet. It has a rather long web address, which you can display here by running the following cell. Clicking the link that appears (htpps://docs.google.com...) will open a new tab with the spreadsheet displayed.

In [ ]:
Markdown(f"Open your spreadsheet in a separate browser tab by clicking on this link: \n \n \
<a href='{viewing_url}' target='_blank'>{viewing_url}</a>")

***
## Step 3: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes 4 separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the combined temperature/humidity sensor, attached to the VINT
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you (pop up) whether you wish to connect to this USB device. Please answer **"yes"** to this security request. 

We now go set up the software to communicate with the Phidget devices.

### 3a. Loading libraries

We need to import libraries for Javascript.

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow some time for the library to load in the background
sleep(1)

### 3b. Opening the USB connection

Steps connecting your USB device to your Phidget VINT device to your computer. 
1. The VINT must be plugged into your computer's USB port. 
1. Click on `▶Run`
1. Follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

This should open a window that asks you to select the Phidget Hub. Do so, and click "OK." The window looks like this:

<img src="images/Pconnect.png" width=300>

In [ ]:
%%js
if (window.usbconn === undefined) {
    element.text("Creating a new USB Connection.");
    window.usbconn = new phidget22.USBConnection();    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

In [ ]:
## We rest for a bit while the USB connects
sleep(1)

### 3c. Confirm the USB connection 

Run the following code to see if the device is connected. 

It should say "true." 

If it does not, check your cable connections. You may also need to check the security settings on your computer to allow new USB devices to get connected.

In [ ]:
%%js
element.text("Is the USB device connected? " + usbconn.connected);

### 3d. Connect the sensors

We make a request to open the four different sensors. Be sure these three devices  are plugged into the VINT device.
1. temperature and humidity sensors (one device)
1. soil moisture 
1. light
 
If you are missing a sensor or two, that is okay 👍🏻. The data collection for the other sensors will still work. The cells below will connect the sensors, then check to see that they are attached.

In [ ]:
%%js

window.tempSensor = new phidget22.TemperatureSensor();
window.humSensor = new phidget22.HumiditySensor();
window.moistSensor = new phidget22.VoltageRatioInput();
window.liteSensor = new phidget22.LightSensor();
    
async function setup_sensors() {
    let errorCode = 0;
    try {await tempSensor.open(1000);} catch {errorCode |= 1;}    
    try {await humSensor.open(1000);} catch {errorCode |= 2;}
    try {await moistSensor.open(1000);} catch {errorCode |= 4;}
    try {await liteSensor.open(1000);} catch {errorCode |= 8;} 
    return errorCode
}

setup_sensors()

In [ ]:
## pause for a bit while the sensors get connected
sleep(10)

### 3e. Confirm the sensor connections

Run the following code cell to see if the four devices are connected. 

The result should be four lines of text that print out the names of the four sensors, and say **"YES"** they are **attached and ready to use**.

If no lines appear, or the answers are **"NO"** it means this is **an error** with the hardware setup. At this point, try checking all the cable connection to the Phidgets and sensors, and start this notebook again. 

If you still are having errors, you may need to restart the kernel or even log out of the Callysto Hub and start again. But normally everything should work just fine.

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))

### 3f. Reading the sensors directly

As a check, you can read the values directly with the following code. 

Let's look at temperature by running the following code cell.  It should print a line indicating what the temperature is. 

(If you get nothing, there is a hardware error. Refer to the instruction in Step 3e above.)

In [ ]:
%%js
try {
    element.text("The temperature is " + tempSensor.temperature);
}
catch {
    element.text("The device is not giving a value.");    
}

The following cells connects the sensors to the gauges, when the update button is pushed. You can push the button to update the gauges.

In [ ]:
## Some code to record the data

def update_gauges(button):
    display(Javascript("""
        if (tempSensor.attached) {
            IPython.notebook.kernel.execute(
                "gauges.data[0]['value'] = " + tempSensor.temperature); }
        if (humSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[1]['value'] = " + humSensor.humidity); }
        if (moistSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[2]['value'] = " + moistSensor.voltageRatio); }
        if (liteSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[3]['value'] = " + liteSensor.illuminance); }
    """))
    

data_button.on_click(update_gauges)

## Step 4: Setup code to display and post the data

The following code grabs data from the gauges and posts it onto the online spreadsheet. In the text boxes, it will also display what data was collected and the result of the online post. Results codes like 200 or 202 are good. 

TIME ZONE CHANGE: If you like, you can change the time zone 'Canada/Pacific' below to your time zone. For instance, for Alberta, change it to 'Canada/Mountain'

The code is connected to a button on the display, so it posts whenever you click the button.

In [ ]:
# set the time zone
tz = pytz.timezone('Canada/Pacific')

# here we update the data results to gauges (g) then post to the spreadsheet. 
def post_stuff(button):
    data = {'Date':datetime.now(tz).strftime("%Y-%m-%d"),
                'Time':datetime.now(tz).strftime("%H:%M:%S"), 
                'Temperature':f'{gauges.data[0]["value"]:.2f}', 
                'Humidity':f'{gauges.data[1]["value"]:.2f}', 
                'Moisture':f'{gauges.data[2]["value"]:.2f}', 
                'Luminance':f'{gauges.data[3]["value"]:.2f}'}
    data_monitor.value = str(list(data.values()))
    r = requests.post(sheet_url, params={'sheet':sheet_name, 'data':json.dumps(data)})
    post_monitor.value = datetime.now(tz).strftime("%Y-%m-%d,%H:%M:%S") + ", Status: " + str(r.status_code)
    
post_button.on_click(post_stuff)

The dashboard is now ready to run. The buttons will now work, to update the gauges and to post data. 

`▶Run` the following cell.

In [ ]:
display(dashboard)

## YAY. We are all done

The data will be updated each time you click the "Update gauges" button. To post this new data, click the "Post to spreadsheet" button. 

You can  see the results in the text boxes, and check the online spreadsheet. The following cell shows the current spreadsheet, which will update when you click the "Post to spreadsheet" button.

In [ ]:
Markdown(f"Open your spreadsheet in a separate browser tab by clicking on this link: \n \n \
<a href='{viewing_url}' target='_blank'>{viewing_url}</a>")

***
## Step 5. Going further with automation

We have the users to manually update the gauges, as this is to make sure the data is up to date Jupyter Hubs. 

But you can automate this and we have the gauges update automatically using a timer in Python but **not always reliable**. 

To have the data continuously updated, we need change the following code  line **"test_code = False" to "test_code = True"**

If you are experiencing issues, you may need to restart the kernel and change the test code back to False.

In [ ]:
test_code = False

def post_loop(g):
    total = 100*24*4 ## post for 100 days (4 per hour gives 100*24*4 total)
    for i in range(total):
        if stop_thread:
            break
        update_gauges(0)
        sleep(2)
        post_stuff(0)
        sleep(15*60-3)  ## Sleep for 15 minutes before next post
        
if test_code:
    stop_thread = False
    thread = threading.Thread(target=post_loop,args=(gauges,))  
    thread.start()

***
## Finishing up. Closing down the sensors

<h2 style="color: red">REALLY IMPORTANT</h2>

Close the sensor now, as otherwise they will keep busy forever, always trying to update the spreadsheet with the latest values. Also, if you just quit the notebook, the sensors may not disconnect properly, which will give you trouble the next time you try to connect.

So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook.

In [ ]:
def doDisconnect(b):
    global stop_thread
    stop_thread = True
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            await moistSensor.close();
            await liteSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)


### Confirm

You can confirm the Phidgets are open or closed by running the following cell. 

If any device is still attached, try clicking the button above, again.

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))

## Conclusion

We have demonstrated the process of running a utility that retrieves Phidget sensor data and publishes it on the web. This operation involves a combination of Javascript and Python to interact with both the front-end and back-end components within the Jupyter notebook environment.

When running this on the [Callysto Hub](https://hub.callysto.ca), it may automatically disconnect after approximately a day of continuous data collection. If you plan to collect data over an extended period, it's essential to have [Jupyter Hub](https://jupyter.org/hub) installed on your local computer.

You may now go to the next notebook to do the data analysis: [plants-analysis.ipynb](plants-analysis.ipynb)

For additional support or inquiries related to this notebook, feel free to reach out to contact@callysto.ca.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)